In [56]:
import csv
from pathlib import Path
from random import randint
import pandas as pd

data_path = Path.cwd().parent.parent / "data"

best_slice_csv_dir=data_path/"17_best_slice_csv"
if not best_slice_csv_dir.exists():
    raise FileNotFoundError(f"Le dossier {best_slice_csv_dir} n'existe pas. Veuillez d'abord exécuter le script de contrast_calculation.ipynb")

lesion_info_csv_dir = data_path / "13_lesion_proportion_with_intensity_csv_dir"
if not lesion_info_csv_dir.exists():
    raise FileNotFoundError(f"Le dossier {lesion_info_csv_dir} n'existe pas. Veuillez d'abord exécuter le 2e script de intensity_extration_per_label.ipynb")

template_txt_dir_path = data_path / "19_2D_best_slice_contrast_intensity_proportion_csv_dir"
if not template_txt_dir_path.exists():
    template_txt_dir_path.mkdir(parents=True, exist_ok=True)

In [57]:
%run "./clinical_parser.ipynb"


id = P1,
sex = F,
timepoint_number = 3,
timepoint_infos = {'T1': <patient_info.TimepointInfo object at 0x11966eb50>, 'T2': <patient_info.TimepointInfo object at 0x15bdc1650>, 'T3': <patient_info.TimepointInfo object at 0x119b94d50>},


id = P2,
sex = M,
timepoint_number = 4,
timepoint_infos = {'T1': <patient_info.TimepointInfo object at 0x119b960d0>, 'T2': <patient_info.TimepointInfo object at 0x119b96950>, 'T3': <patient_info.TimepointInfo object at 0x119bbc5d0>, 'T4': <patient_info.TimepointInfo object at 0x119bbca50>},


id = P3,
sex = F,
timepoint_number = 4,
timepoint_infos = {'T1': <patient_info.TimepointInfo object at 0x119bbd950>, 'T2': <patient_info.TimepointInfo object at 0x119bbd850>, 'T3': <patient_info.TimepointInfo object at 0x119bbdb50>, 'T4': <patient_info.TimepointInfo object at 0x119bbd9d0>},


timepoint = T1,
age = 28.09,
ms_type = SMRR,
edss = 3.5,
lesion_number = 18,
lesion_volume = 20674.0


timepoint = T2,
age = 32.1,
ms_type = SMRR,
edss = 0.0,
lesion_number = 

In [58]:
def read_csv_dict(csv_path):
    """Lit un CSV et renvoie une liste de dicts (chaque ligne)."""
    with csv_path.open("r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        return [row for row in reader]

In [73]:
"""
11 and 13 -> 16
Générer les templates sous forme de {patient_id}_{timepoint}_{modalité}_{lesion_id}_3D_Description
"""

def print_lesion_text(patient_id, timepoint, lesion_id,lesion_text):
    path_morpho = data_path /"19_2D_best_slice_contrast_intensity_proportion_csv_dir"/ f"{patient_id}"/"02_morphology.csv"
    path_signal = data_path /"19_2D_best_slice_contrast_intensity_proportion_csv_dir"/ f"{patient_id}"/"03_signal_contrast.csv"
    if path_morpho.exists() and path_signal.exists():
        morphology_df = pd.read_csv(path_morpho)
        signal_df = pd.read_csv(path_signal)

        morpho_vals = get_lesion_values_auto(
            path_morpho,
            lesion_id,
            ["major_axis_mm", "minor_axis_mm"]
        )

        signal_vals = get_lesion_values_auto(
            path_signal,
            lesion_id,
            ["lesion_mean", "neighborhood_mean", "contrast"]
        )

        major_axis_mm = morpho_vals["major_axis_mm"]
        minor_axis_mm = morpho_vals["minor_axis_mm"]

        lesion_mean = signal_vals["lesion_mean"]
        neighborhood_mean = signal_vals["neighborhood_mean"]
        contrast = signal_vals["contrast"]

        lesion_text += (
        f"Major axis is {major_axis_mm} mm and minor axis is {minor_axis_mm} mm.\n"
        f"Lesion mean is {lesion_mean}, neighborhood mean is {neighborhood_mean}, "
        f"contrast is {contrast}.\n"
        )
        return lesion_text
    else:
        print(f"[WARN] Texte manquant")

def get_lesion_values_auto(csv_path, lesion_id, columns):
    lid_str = str(lesion_id)
    lid_int = int(lesion_id)
    row_id = f"L{lesion_id}"

    with open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        rows = list(reader)
        if not rows:
            raise ValueError(f"Empty CSV: {csv_path}")

        first_col = reader.fieldnames[0]

        # 1) Cas où la 1ère colonne contient "L1", "L2", ...
        for r in rows:
            if r.get(first_col) == row_id:
                return {c: float(r[c]) for c in columns}

        # 2) Cas où il y a une colonne "lesion_id"
        if "lesion_id" in reader.fieldnames:
            for r in rows:
                if str(r.get("lesion_id")) == lid_str:
                    return {c: float(r[c]) for c in columns}

        # 3) Fallback: prendre par index de ligne (0-based ou 1-based)
        if 0 <= lid_int < len(rows):
            r = rows[lid_int]
            return {c: float(r[c]) for c in columns}
        if 1 <= lid_int <= len(rows):
            r = rows[lid_int - 1]
            return {c: float(r[c]) for c in columns}

        raise ValueError(
            f"Cannot find lesion {lesion_id} in {csv_path}. "
            f"Tried row id '{row_id}', column 'lesion_id', and row positions."
        )


# Parcours des fichiers CSV de lésion
for file_path in lesion_info_csv_dir.iterdir():
    if not file_path.name.endswith("_lesion_proportion.csv"):
        continue

    parts = file_path.stem.split("_")
    if len(parts) == 5:
        patient_id, timepoint, modality = parts[0], parts[1], parts[2]
    elif len(parts) == 4:
        patient_id, timepoint, modality = parts[0], "T1", parts[1]
    else:
        print(f"[SKIP] Nom de fichier non conforme : {file_path.name}")
        continue

    lesion_rows = read_csv_dict(file_path)

    best_slice_csv_path = best_slice_csv_dir / f"{patient_id}_best_slice_all_lesions.csv"
    best_slice_rows = read_csv_dict(best_slice_csv_path) if best_slice_csv_path.exists() else []


    # Récupérer infos patient
    current_patient = next((p for p in patients if p.id == patient_id), None)
    allowed_patients = [f"P{i}" for i in range(1, 54)]


    if current_patient.id not in allowed_patients:
        continue

    if current_patient is None:
        raise ValueError(f"Patient {patient_id} non trouvé.")

    sex = "male" if current_patient.sex == "M" else "female" if current_patient.sex == "F" else "unknown"
    timepoint_info = current_patient.timepoint_infos.get(timepoint)
    age = int(timepoint_info.age) if timepoint_info else "unknown"

    for lesion in lesion_rows:
        lesion_id = lesion["lesion_id"]

        # Intensité et volume
        volume = float(lesion.get("lesion_volume_mm3", 0))
        mean_int = float(lesion.get("lesion_mean", 0))
        min_int = float(lesion.get("lesion_min", 0))
        max_int = float(lesion.get("lesion_max", 0))

        # Contraste
        bs = next(
            (r for r in best_slice_rows if str(r.get("lesion_id")) == str(lesion_id)),
            None
            )

        best_slice_z = int(float(bs["best_slice_z"])) if bs else "unknown"
        best_slice_y = int(float(bs["best_slice_y"])) if bs else "unknown"
        best_slice_x = int(float(bs["best_slice_x"])) if bs else "unknown"

        centroid_x = float(bs["centroid_x"]) if bs else "unknown"
        centroid_y = float(bs["centroid_y"]) if bs else "unknown"
        centroid_z = float(bs["centroid_z"]) if bs else "unknown"

        best_area_z_px = float(bs["best_area_z_px"]) if bs else "unknown"
        best_area_y_px = float(bs["best_area_y_px"]) if bs else "unknown"
        best_area_x_px = float(bs["best_area_x_px"]) if bs else "unknown"

        flatness_3d = float(bs["flatness_3d"]) if bs else "unknown"
        density_3d  = float(bs["density_3d"]) if bs else "unknown"

        # Localisation anatomique
        prop_cols = [k for k in lesion.keys() if k.endswith("_prop")]
        prop_values = [(k, float(lesion[k])) for k in prop_cols if float(lesion[k]) > 0]
        prop_values.sort(key=lambda x: x[1], reverse=True)
        zones_text = [f"{k.replace('_prop','').replace('_',' ')} ({v*100:.1f}%)" for k,v in prop_values]
        zones_str = ", ".join(zones_text)

        # Texte final
        prefixes = [
            f"This {modality} scan corresponds to a {age}-year-old {sex} subject {patient_id} at timepoint {timepoint}. ",
            f"This is a {modality} scan of a {sex} subject {patient_id}, aged {age}, timepoint {timepoint}. ",
            f"This {modality} scan shows a {sex} subject {patient_id} aged {age} of timepoint {timepoint}. ",
            f"A {modality} scan of a {sex} subject {patient_id} at timepoint {timepoint}, {age} years old. ",
            f"Here is a {modality} scan of a {sex} subject {patient_id} at timepoint {timepoint} aged {age}. ",
            f"This {modality} scan captures the brain of a {sex} subject {patient_id} aged {age}, timepoint {timepoint}. ",
            f"A {modality} scan from a {sex} subject {patient_id}, {age} years old, at timepoint {timepoint}. ",
            f"This {modality} scan, taken from a {sex} subject {patient_id} aged {age} at timepoint {timepoint}. ",
            f"This {modality} scan provides a view of the brain of a {age} subject {patient_id}, aged {age}. "
        ]
        lesion_text = prefixes[randint(0, len(prefixes)-1)] + (
            f"The best slice coordinates are (z={best_slice_z}, y={best_slice_y}, x={best_slice_x}).\n"
            f"The centroid is at (x={centroid_x}, y={centroid_y}, z={centroid_z}).\n"
            f"Best areas (px) are z={best_area_z_px}, y={best_area_y_px}, x={best_area_x_px}.\n"
            f"Flatness (3D) is {flatness_3d} and density (3D) is {density_3d}.\n"
        )
        lesion_text=print_lesion_text(patient_id, timepoint, lesion_id,lesion_text)

        # Sauvegarde
        filename = f"{patient_id}_{timepoint}_{modality}_L{lesion_id}_2D_description.txt"
        output_path = template_txt_dir_path / filename
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with output_path.open("w", encoding="utf-8") as f:
            f.write(lesion_text)